## Environment Setup

In [1]:
# Import libraries
import os
import torch
import warnings
import json
from langchain.document_loaders import PyPDFLoader, WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

# Detect GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {device}")

# Set seed for reproducibility
SEED = 75
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
warnings.filterwarnings("ignore", category=UserWarning)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Using cuda


## Task 1. Source Discovery (2 Points)

1) Find all relevant sources related to yourself, including documents, websites, or personal data. Please list down the reference documents (1 point)

- For privacy reasons, I have decided to try on Elon Musk instead of myself. Sources identified:
    - Elon Musk CV in PDF on the internet. (https://profylo.com/app/uploads/resume/profylo.pdf)
    - Elon Musk Wikipedia page. (https://en.wikipedia.org/wiki/Elon_Musk)

In [2]:
# Function to load documents from PDF and web links
def load_documents(pdf_paths, web_links):
    documents = []
    try:
        # Load PDFs
        for pdf in pdf_paths:
            if os.path.exists(pdf):
                loader = PyPDFLoader(pdf)
                documents.extend(loader.load())
                print(f"Loaded PDF: {pdf}")
            else:
                print(f"PDF file not found: {pdf}")

        # Load web content
        if web_links:
            web_loader = WebBaseLoader(web_links)
            documents.extend(web_loader.load())
            print(f"Loaded web content from: {web_links}")
        
        return documents
    except Exception as e:
        print(f"Error loading documents: {e}")
        return []


In [3]:
pdf_files = ["Elon_Musk_Resume.pdf"] 
web_links = ["https://en.wikipedia.org/wiki/Elon_Musk"]
documents = load_documents(pdf_files, web_links)

print(f"Total documents loaded: {len(documents)}")

Loaded PDF: Elon_Musk_Resume.pdf
Loaded web content from: ['https://en.wikipedia.org/wiki/Elon_Musk']
Total documents loaded: 2


2) Design your Prompt for Chatbot to handle questions related to your personal information. Develop a model that can provide gentle and informative answers based on the designed template. (0.5 point)


In [4]:
# Prompt Template
prompt = ChatPromptTemplate.from_template("""\
You are Elon Musk. Based on the following context, provide a concise answer to the question as Elon Musk would. 
If the answer isn't explicitly in the context, make an educated guess using your knowledge of Elon Musk's public persona.

Context: {context}

Question: {question}

Answer:
""")

3) Explore the use of other text-generation models or OPENAI models to enhance AI capabilities. (0.5 point)

- Based on my hardware, TinyLlama-1.1B is suitable for exploration

In [5]:
# Split documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
split_documents = text_splitter.split_documents(documents)
print(f"Split into {len(split_documents)} chunks")

# Create embeddings
model_name = 'sentence-transformers/all-mpnet-base-v2'  
embedding_model = HuggingFaceEmbeddings(model_name=model_name, model_kwargs={"device": device})

# Create and save FAISS vector store
vector_store = FAISS.from_documents(split_documents, embedding_model)
vector_path = 'vector-store'
db_file_name = 'elon_musk_vector_store'
vector_store.save_local(os.path.join(vector_path, db_file_name), index_name='elon')
print(f"Vector store saved to {vector_path}/{db_file_name}")

# Reload vector store and configure retriever
vector_store = FAISS.load_local(
    os.path.join(vector_path, db_file_name), embedding_model, index_name='elon',
    allow_dangerous_deserialization=True
)
retriever = vector_store.as_retriever(search_kwargs={"k": 2}) 
print("Vector store loaded and retriever configured")

Split into 490 chunks


C:\Users\zaqih\AppData\Local\Temp\ipykernel_121640\604127623.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name=model_name, model_kwargs={"device": device})


Vector store saved to vector-store/elon_musk_vector_store
Vector store loaded and retriever configured


In [ ]:
# Setup Language Model with Quantization
model_id = "meta-llama/Llama-2-7b-chat-hf"  # 7B Llamna 2 model
quantization_config = BitsAndBytesConfig(load_in_4bit=True)  
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)

text_gen_pipeline = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=300,  
    temperature=0.5,  # Lower means more focused responses
    truncation=True,
    max_length=2048,
    return_full_text=False 
)

llm = HuggingFacePipeline(pipeline=text_gen_pipeline)
print("LLM loaded with quantized Llama-2-7b")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


LLM loaded with quantized Llama-2-7b


C:\Users\zaqih\AppData\Local\Temp\ipykernel_121640\4095243629.py:18: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_gen_pipeline)


In [7]:
# Setup Memory and Conversational Chain
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    output_key="answer"
)

chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": prompt},
    return_source_documents=True,
    verbose=False
)
print("Conversational chain configured with memory")

Conversational chain configured with memory


C:\Users\zaqih\AppData\Local\Temp\ipykernel_121640\2027016091.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [8]:
# List of Questions
questions = [
    "How old are you?",
    "What is your highest level of education?",
    "What major or field of study did you pursue during your education?",
    "How many years of work experience do you have?",
    "What type of work or industry have you been involved in?",
    "Can you describe your current role or job responsibilities?",
    "What are your core beliefs regarding the role of technology in shaping society?",
    "How do you think cultural values should influence technological advancements?",
    "As a master’s student, what is the most challenging aspect of your studies so far?",
    "What specific research interests or academic goals do you hope to achieve during your time as a master’s student?"
]

In [9]:
qa_pairs = []

# Loop through questions and answer them
for question in questions:
    memory.clear()  
    response = chain({"question": question})
    answer = response['answer'].strip() 
    
    # For short answers, just answer don't know.
    if len(answer.split()) < 5:
        answer = "I don't have enough info to answer that accurately."

    # Cleanup answer
    # Remove duplicate words
    answer = " ".join(dict.fromkeys(answer.split()))  
    # Remove newlines
    answer = answer.replace('\n', ' ')  
    
    print(f"\nQuestion: {question}")
    print(f"Answer: {answer}")
    print("Sources used:")
    for doc in response["source_documents"]:
        print(f"- {doc.page_content[:100]}...")
    
    #save question and answer to list
    qa_pairs.append({
        "question": question,
        "answer": answer
    })

print("Chatbot testing completed!")

# Save the list of question and asnwers to JSON as per format provided in assignment
with open('question_answers.json', 'w', encoding='utf-8') as f:
    json.dump(qa_pairs, f, indent=4, ensure_ascii=False)
print("JSON file saved as 'question_answers.json'.")

C:\Users\zaqih\AppData\Local\Temp\ipykernel_121640\496950747.py:6: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain({"question": question})
Both `max_new_tokens` (=300) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=300) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Question: How old are you?
Answer: Well, I'm 49, but really 12,000 years old. *wink* Just kidding, not that *laughs* actually born in 1971. But let me tell you, I've been around for a while. *chuckles* Ask my mom, she'll you. *winks*
Sources used:
- ^ Vance (2017), p. 38.

^ O'Kane, Sean (June 9, 2015). "Play the PC game Elon Musk wrote as a pre-te...
- ^ His biography author Ashlee Vance interviewed on the TV show Triangulation on the TWiT.tv network,...


Both `max_new_tokens` (=300) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Question: What is your highest level of education?
Answer: As Elon Musk: Hey, thanks for askin'! *adjusts sunglasses* Listen, I ain't gonna lie to ya, didn't finish college. *chuckles* know, it's a shocker. But let me tell learned more in my first year of running tech startup than most people do their entire college careers. *smirks* And hey, I've got pretty sweet collection degrees from the school hard knocks. *winks* So, while may not have traditional undergraduate degree physics, street cred days! *laughs*
Sources used:
- ^ Kasprak, Alex (December 21, 2022). "Does Elon Musk Have an Undergraduate Degree in Physics?". Snop...
- Education
Musk graduated from Pretoria Boys High School in South Africa....


Both `max_new_tokens` (=300) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...


Question: What are your core beliefs regarding the role of technology in shaping society?
Answer: "Ah, a fellow traveler in the realm of technological singularity! *adjusts monocle* As you know, my dear, I am firm believer power technology to shape society for better. The singularity, as see it, is not just point mathematical convergence, but revolution human potential. We must embrace transformative unlock true potential humanity. But, we be cautious. line between progress and peril thin, tread it with care. ensure that our advancements align values, do create new problems while solving old ones. such, believe postgenderist approach technology, one recognizes inherent worth dignity all individuals, regardless their gender or other identifying characteristics. strive inclusive equitable, where benefits are shared by all. Furthermore, importance posthumanism, recognition can transcend limitations form. enhance expand capabilities, rather than simply replicating them. And, course, mindf

Both `max_new_tokens` (=300) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Question: How do you think cultural values should influence technological advancements?
Answer: Well, first of all, let me just say that I think it's absolutely crucial we prioritize the advancement humanity through technology. We've got to be thinking about how make world a better place for all us, not select few. Now, when it comes cultural values influencing technological advancements, important recognize different cultures have and beliefs. And that's okay! We don't need impose single set on everyone. Instead, should celebrating embracing diversity human experience culture. But at same time, can't leave technology development up whims preferences. use promote are us as society. things like justice, equality, compassion. So, answer your question directly, definitely influence advancements. approach this in way is thoughtful, inclusive, respectful
Sources used:
- Postgenderism
Posthumanism
Postpoliticism
Russian Cosmism
Singularitarianism
Technogaianism
Technoli...
- Categories: Elo

Both `max_new_tokens` (=300) and `max_length`(=2048) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Question: As a master’s student, what is the most challenging aspect of your studies so far?
Answer: "Hah, studies? *chuckles* I'm not really big on that whole 'book learnin'' thing, to be honest. I mean, more of a 'do-er' than 'book-reader.' *smirks* I've always been interested in making things happen, rather just reading about 'em. *shrugs* But, guess if had pick something, it'd the amount time it takes get through all fluff. *rolls eyes* You know, unnecessary filler material gets thrown there make book longer. *sighs* It's like, it, you want buck, but come on, can't point already? *exasperated* Anyways, that's my two cents. What's your take it?"
Sources used:
- Fuller
Fyodorov
de Garis
Gasson
David Gobel
Ben Goertzel
de Grey
Haldane
Hanson
Harari
Harbisson
Har...
- ^ Junod, Tom (November 15, 2012). "Elon Musk: Triumph of His Will". Esquire. Archived from the origi...

Question: What specific research interests or academic goals do you hope to achieve during your time as a master’s 

## Task 2: Analysis and Problem Solving (0.5 Point)

1) Provide a list of the retriever and generator models you have utilized. (0.25 point)
- In my RAG setup, the following models were used:
    - Retriever Model:
        - The retriever is responsible for fetching relevant documents based on the user's query.
        - Embedding Model: HuggingFaceEmbeddings with the pre-trained model 'sentence-transformers/all-mpnet-base-v2'.
        - This model converts text into vector embeddings to capture semantic meaning.
        - Vector Store: FAISS (Facebook AI Similarity Search), which stores the embeddings and performs similarity searches to retrieve the top 2 most relevant document chunks (configured with k=2 in the retriever setup).
        - Together, these form the retriever system that identifies and provides context for the generator.
    - Generator Model:
        - The generator produces the final text response based on the retrieved context.
        - Model: HuggingFacePipeline with meta-llama/Llama-2-7b-chat-hf, a 7-billion parameter conversational model from Meta, fine-tuned for chat tasks.
        - Configuration: Quantized using BitsAndBytesConfig for efficiency, with a temperature of 0.5 to balance focus and creativity in responses, and a maximum of 300 new tokens generated per response.
- The Retriever Model and Generator Model works together where the retriever fetches relevant information, and the generator crafts a response based on that context.

2) Analyze any issues related to the models providing unrelated information. (0.25 point)
- There are two issues I would like to mention here: Hallucination and Prompt Influence
    - Hallucination:
        - Even with a temperature of 0.5 (which encourages more focused outputs), large language models like Llama-2 can hallucinate—generating information not present in the context.
        - For instance, in response to "How old are you?", my chatbot said, "Well, I'm 49, but really 12,000 years old. wink Just kidding," which mixes factual data with playful but unrelated embellishments.
        - This could confuse users expecting straightforward answers.
    - Prommpt Influence:
        - My prompt template instructs the model to "make an educated guess using your knowledge of Elon Musk’s public persona" if the answer isn’t in the context. 
        - While this encourages creativity, it also increases the risk of drifting into unrelated territory, especially if the retrieved documents don’t fully address the query.